In [5]:
import numpy as np
import pandas as pd
import json
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns
import re
import glob
import os

from scipy import stats
from scipy.integrate import trapz

import missingno as msno

import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 10, 'max_rows', 5, 'max_colwidth', 10)

In [ ]:
## 데이터 셋 만들기
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold

## 데이터 전처리
from sklearn.decomposition import TruncatedSVD, PCA, NMF
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis # 차원축소
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import Binarizer
from numpy.linalg import svd # 행렬분해
from scipy.sparse.linalg import svds # 행렬분해
from scipy.linalg import svd # 행렬분해
from sklearn.preprocessing import LabelBinarizer # 원-핫 인코딩 변환 후 희소행렬 최적화 형태로 저장
from  scipy.sparse import hstack # 원-핫 인코딩된 희소 행렬을 모두 scipy 패키지의 hstack()함수를 이용하여 결합
import gc
import nltk
import string
from nltk import sent_tokenize # 문장 토큰화
from nltk import word_tokenize # 단어 토큰화
from nltk import word_tokenize, sent_tokenize # 여러 문장들에 대한 단어 토큰화
from nltk import pos_tag
from nltk import ngrams # n-gram
import nltk
nltk.download('stopwords') # stopwords 제거
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.decomposition import LatentDirichletAllocation # LDA는 Count기반 Vectorizer만 적용

from ast import literal_eval # 텍스트 문자 1차 가공, 파이썬 딕셔너리 변환 후 리스트 형태로 변환

## 알고리즘(분류)
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import BotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from xgboost import plot_importance

from lightgbm import LGBMClassifier
from lightgbm import plot_importance

## 알고리즘(군집)
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth # 최적의 bandwidth값을 estimate_bandwidth()로 계산 한 뒤에 다시 군집화 수행
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture

## 알고리즘(회귀)
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import statsmodels.api as sm

## 알고리즘(텍스트 마이닝)
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer


## 데이터 평가
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score , recall_score
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.metrics import precision_recall_curve
from sklearn.metrics.pairwise import cosine_similarity

from scipy.stats import skew


## 기타 모르겠는 알고리즘
from sklearn.base import BaseEstimator
from sklearn.tree import export_graphviz
import graphviz
import matplotlib.ticker as ticker
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE # 오버샘플링

# surprise 패키지
from surprise import Reader # load_form_file()을 이용하여 데이터셋 로딩
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate # cross_validate()를 이용한 교차 검증
from surprise.dataset import DatasetAutoFolds # TrainSet클래스 변환

In [ ]:
# csv 로드
csv_df = pd.read_csv('./data.csv')

# txt 로드(https://rfriend.tistory.com/523)
with open('.text.txt', 'r', encoding='utf-8', errors='ignore') as f:
    text_string = f.read()

## 분류

## 함수로 만들고 들어가서는 그 함수를 적고 다로 적용할 수 있도록 나눠서 만든다.(20분 내로 적어서 만들 수 있게 함)

In [ ]:
# 데이터 전처리
# method : 'Standard', 'Minma', 'Log'
# l_degree : 2,3,4
# input_data : 
def get_scaled_data(method='None', p_degree=None, input_data=None):
    if method == 'Standard':
        scaled_data = StandardScaler().fit_transform(input_data)
    elif method == 'MinMa':
        scaled_data = MinMaxScaler().fit_transform(input_data)
    elif method == 'Log':
        scaled_data = np.log1p(input_data)
    else:
        scaled_data = input_data
        
    if p_degree != None:
        scaled_data = PolynomialFeatures(degree=p_degree, include_bias=False).fit_transform(scaled_data)
        
    return scaled_data

## 시나리오를 작성해서 만들어야 함(분류/군집/회귀/)
데이터 전처리
- 예를 들면 스케일링할때 첨도가 높으면 로그를 씌운다.
- null값이 있으면 빼던가 평균으로 대체한다.
- 이상치가 있으면 그 데이터를 삭제 후 재 학습/예측/평가
- 시계열은 함수로 만들어져 있는게 있으니 그걸 가져다 쓴다고 생각하자

staking도 해볼까?

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform()
X_test = scaler.fit_transform()

In [ ]:
ran = RandomForestClassifier(random_state=1)
knn = KNeighborsClassifier()
log = LogisticRegression()
xgb = XGBClassifier()
gbc = GradientBoostingClassifier()
svc = SVC(probability=True)
ext = ExtraTreesClassifier()
ada = AdaBoostingClassifier()
gnb = GaussianNB()
gpc = GaussianProcessClassifier()
bag = BaggingClassifier()

models = [ran, knn, log, xgb, gbc, svc, ext, ada, gnb, gpc, bag]
model_names = ['Random Forest', 'K Nearest Neighnour', 'Logistic Regression', 'XGBoost', 'Gradient Boosting', 'SVC', 'Extra Trees', 'AdaBoost', 'Gaussian Naive Bayes', 'Gaussian Process', 'Bagging Classifier']
scores = {}
               
for ind, mod in enumerate(models):
    mod.fit(X_train, y_train)
    acc = cross_val_score(mod, X_train, y_train, scoring="accuracy", cv=10)
    scores[model_names[ind]] = acc